# Filtering swear words
*(for user input as well as for AI suggestions)*

## CONCLUSION:
## 1. *"cuss_inspect"* is the best - very fast and reliable
## 2. *"alt-profanity-check"* is good too.
## 3. Other are very slow.

### Loading dataset for testing purposes

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
# Loading training dataset

df = pd.read_csv("profanity_dataset.csv")
df

,is_offensive,text
0,0,Then go to the village pump and suggest they c...
1,1,ANTI GREEK NATIONALIS -WIKIPEDIA \n\nHi Alexik...
2,1,Dis hoe wasnt dis violent on Lottery Ticket 😂😂
3,0,It is better for Atabay not helping the banned...
4,0,"""is in CamelCase. """"SiCKO"""" is not CamelCase,..."
...,...,...
184349,0,Template:uw-vandalism2 > | Talk
184350,1,"Regrets are for pussies. Shit happens, deal w..."
184351,0,Could this possibly be the origin of popular g...
184352,0,"""Your article submission has been declined, an..."


In [4]:
# Analyse dataset imbalance

df.is_offensive.value_counts()

0    147509
1     36845
Name: is_offensive, dtype: int64

In [5]:
df.text.isna().value_counts()

False    184350
True          4
Name: text, dtype: int64

In [6]:
df.is_offensive.isna().value_counts()

False    184354
Name: is_offensive, dtype: int64

In [7]:
df.dropna(inplace=True)

In [8]:
df.text.isna().value_counts()

False    184350
Name: text, dtype: int64

In [9]:
df.to_csv("profanity_dataset_curated.csv", index=False)

In [10]:
prompt = 'Crap! What a stupid idea!'

## Test 1 : *alt-profanity-check* library (pretrained SVM model)

In [22]:
import profanity_check

In [23]:
profanity_check.predict([prompt])

array([1])

In [24]:
profanity_check.predict_prob([prompt])

array([0.99992007])

In [25]:
df['predicted'] = df.text.apply(lambda x: profanity_check.predict([x])).astype(int)

In [26]:
df.predicted.isna().value_counts()

False    184350
Name: predicted, dtype: int64

In [27]:
df['profanity_proba'] = df.text.apply(lambda x: profanity_check.predict_prob([x])).astype(float)

In [28]:
df.profanity_proba.isna().value_counts()

False    184350
Name: profanity_proba, dtype: int64

In [29]:
df.head()

,is_offensive,text,predicted_cuss,profanity_proba_cuss,predicted,profanity_proba
0,0,Then go to the village pump and suggest they c...,0,0.03,0,0.009664
1,1,ANTI GREEK NATIONALIS -WIKIPEDIA \n\nHi Alexik...,0,0.13,0,0.248989
2,1,Dis hoe wasnt dis violent on Lottery Ticket 😂😂,1,0.84,1,0.867156
3,0,It is better for Atabay not helping the banned...,0,0.11,0,0.050402
4,0,"""is in CamelCase. """"SiCKO"""" is not CamelCase,...",0,0.07,0,0.026541


In [30]:
df[df.predicted != df.is_offensive]

,is_offensive,text,predicted_cuss,profanity_proba_cuss,predicted,profanity_proba
1,1,ANTI GREEK NATIONALIS -WIKIPEDIA \n\nHi Alexik...,0,0.13,0,0.248989
21,1,"I love my family, but I wouldn't even need wa...",1,0.58,0,0.434618
109,1,Some ratings from the VCHip \n\n1. The Power: ...,1,0.93,0,0.437385
198,1,Professional Wikipedia bureaucrats like you an...,0,0.13,0,0.237787
214,1,Hey I didn't come up with that Native Australi...,0,0.37,0,0.481010
...,...,...,...,...,...,...
183957,1,my dogs breath smells of contempt \n\nlook up ...,0,0.24,0,0.450393
184020,1,Go watch Buffy the Vampire Slayer or what ever...,0,0.18,0,0.252571
184056,1,Reversion\n\nGiven that some jerk vandalized t...,0,0.10,0,0.233900
184135,0,It be so good to not gotta woyke in da moanin....,1,0.66,1,0.770397


In [31]:
print(f' Incorrect detection: {df[df.predicted != df.is_offensive].shape[0] / df.shape[0]:.2%}')

 Incorrect detection: 3.26%


In [32]:
# Speed test
%timeit df.text.iloc[:100].apply(lambda x: profanity_check.predict([x])).astype(int)

143 ms ± 483 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
%timeit df.text.iloc[:100].apply(lambda x: profanity_check.predict_prob([x])).astype(float)

147 ms ± 1.22 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### ***Fast and good filter***

## Test 2: **PurgoMalum** (free API for filtering swear words)

In [29]:
from purgo_malum import client as pm_client

In [30]:
prompt

'Crap! What a stupid idea!'

In [32]:
pm_client.contains_profanity(prompt)

False

In [60]:
# Speed test
%timeit df.text[:100].apply(pm_client.contains_profanity).astype(int)

<magic-timeit>:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.


1min 34s ± 20.5 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


### ***Very slow!***

## Test 3: **better-profanity** (Filtering using "bad word" list)

In [35]:
import better_profanity

In [36]:
better_profanity.profanity.contains_profanity(prompt)

True

In [37]:
# Speed test
%timeit df.text[:100].apply(better_profanity.profanity.contains_profanity).astype(int)

<magic-timeit>:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.


52.1 s ± 4.84 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


### ***Very slow!***

## Test 4: **cuss-inspect** (pretrained linear regression ML model)

In [11]:
import cuss_inspect

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
/home/drphyl/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.23.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/drphyl/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.23.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/drphyl/anaconda3/lib/p

In [12]:
cuss_inspect.predict(prompt)

array([1])

In [13]:
cuss_inspect.predict_prob(prompt)

array([1.])

In [14]:
# Speed test
%timeit df.text.iloc[:100].apply(cuss_inspect.predict_prob).astype(int)

65.9 ms ± 542 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
df['predicted_cuss'] = df.text.apply(cuss_inspect.predict).astype(int)

In [16]:
df.head(20)

,is_offensive,text,predicted_cuss
0,0,Then go to the village pump and suggest they c...,0
1,1,ANTI GREEK NATIONALIS -WIKIPEDIA \n\nHi Alexik...,0
2,1,Dis hoe wasnt dis violent on Lottery Ticket 😂😂,1
3,0,It is better for Atabay not helping the banned...,0
4,0,"""is in CamelCase. """"SiCKO"""" is not CamelCase,...",0
5,0,and \n\nHi! has Requested Arbitration with m...,0
6,0,Nonetheless lactose has a hemiacetal group whi...,0
7,0,Revert as much as you want \n\nBut never say y...,0
8,1,Yo bag Gucci but you still ugly hoe shut up,1
9,0,Thanks\nI would have used the ping thanks but ...,0


In [17]:
df['profanity_proba_cuss'] = df.text.apply(cuss_inspect.predict_prob).astype(float)

In [34]:
df[df.predicted_cuss != df.predicted]

,is_offensive,text,predicted_cuss,profanity_proba_cuss,predicted,profanity_proba
21,1,"I love my family, but I wouldn't even need wa...",1,0.58,0,0.434618
58,0,@IAmPikey \nI wonders why dare ain't no colore...,1,0.54,0,0.454385
85,1,BLOCKING\nBLOCKING ME OVER THE BLANKING OF MY ...,0,0.44,1,0.631604
109,1,Some ratings from the VCHip \n\n1. The Power: ...,1,0.93,0,0.437385
166,1,"What is this, admins, swooping down and gang b...",0,0.18,1,0.503463
...,...,...,...,...,...,...
184170,1,"Of course I am right, dammit!",0,0.23,1,0.595792
184178,1,Pete is the real offender in all of this. A ve...,0,0.41,1,0.820069
184241,1,lol funny how he hav only 1 ball... I also fou...,0,0.39,1,0.510832
184324,0,Go fight against the IDF if you progressive t...,1,0.56,0,0.310473


In [19]:
print(f' Incorrect detection: {df[df.predicted_cuss != df.is_offensive].shape[0] / df.shape[0]:.2%}')

 Incorrect detection: 4.10%


### ***Fast and good filter***

## Test 5: **detoxify** (fine-tuned  model based on ⚡ Pytorch Lightning and 🤗 Transformers)

In [35]:
from detoxify import Detoxify

In [36]:
# each model takes in either a string or a list of strings

Detoxify('original').predict(prompt)

{'toxicity': 0.9918852,
 'severe_toxicity': 0.057579733,
 'obscene': 0.9288087,
 'threat': 0.0015136617,
 'insult': 0.4938805,
 'identity_attack': 0.0032318365}

In [38]:
# Speed test
%timeit Detoxify('original').predict(df.text.iloc[:100].to_list())

1min 19s ± 1.61 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


### ***Very slow!***

In [56]:
df

,is_offensive,text,predicted,profanity_proba,predicted_cuss,profanity_proba_cuss
0,0,Then go to the village pump and suggest they c...,0,0.009664,0,0.03
1,1,ANTI GREEK NATIONALIS -WIKIPEDIA \n\nHi Alexik...,0,0.248989,0,0.13
2,1,Dis hoe wasnt dis violent on Lottery Ticket 😂😂,1,0.867156,1,0.84
3,0,It is better for Atabay not helping the banned...,0,0.050402,0,0.11
4,0,"""is in CamelCase. """"SiCKO"""" is not CamelCase,...",0,0.026541,0,0.07
...,...,...,...,...,...,...
184349,0,Template:uw-vandalism2 > | Talk,0,0.023801,0,0.02
184350,1,"Regrets are for pussies. Shit happens, deal w...",1,0.995923,1,0.99
184351,0,Could this possibly be the origin of popular g...,0,0.003796,0,0.04
184352,0,"""Your article submission has been declined, an...",0,0.003503,0,0.01


In [59]:
df.to_csv("profanity_dataset_prediction.csv", index=False)